# download, prepopulate all the csv into minio

In [34]:
import pandas as pd
# import pyarrow as pa
# import pyarrow.parquet as pq
import pandas as pd
import dask.dataframe as dd
from dask import delayed

In [2]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
from dask.distributed import Client, LocalCluster
import dask
from dask.distributed import get_task_stream

In [3]:
import time

In [4]:
print('pandas version: %s' % pd.__version__)
print('numpy version: %s' % np.__version__)
print('dask version: %s' % dask.__version__)

pandas version: 1.3.5
numpy version: 1.22.2
dask version: 2021.09.1


https://docs.databricks.com/_static/notebooks/koalas-benchmark-distributed-execution.html?_ga=2.216403934.95291449.1648935555-599276868.1645477063

In [5]:
filename = "taxi_dataset_2009-13.txt"

In [6]:
with open(filename) as file:
    csv_files = [line.rstrip() for line in file]
# only choose yellow taxis
yellow = list(filter(lambda x: "yellow" in x, csv_files))

In [7]:
# make the list small for now
# yellow = ['https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-11.csv']

In [8]:
# yellow

# Configure minio

In [10]:
import getpass

In [11]:


#tag::minio_storage_options[]
# straight from holden's example
minio_storage_options = {
    "key": getpass.getpass(),
    "secret": getpass.getpass(),
    "client_kwargs": {
        # "endpoint_url": "http://minio-1602984784.minio.svc.cluster.local:9000",
        # "endpoint_url": "http://minio.pigscanfly.ca:9000",
        "endpoint_url": "http://10.43.207.163:9000",
        "region_name": 'us-east-1'
    },
    "config_kwargs": {"s3": {"signature_version": 's3v4'}},
}
#end::minio_storage_options[]

 ····
 ···············································


In [8]:
# !pip install pyarrow

  Using cached pyarrow-7.0.0-cp38-cp38-manylinux_2_17_aarch64.manylinux2014_aarch64.whl (24.9 MB)


In [12]:
# !conda install -c conda-forge s3fs

In [25]:
# !pip install fsspec[s3] aiohttp

In [12]:
# import pyarrow

['https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-01.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-02.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-03.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-04.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-05.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-06.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-07.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-08.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-09.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-10.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-11.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-12.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2012-01.csv',
 'https://s3.amazonaws.co

In [11]:
len(yellow)

60

In [12]:
# yellow = yellow[:3]

In [13]:
# yellow

['https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-01.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-02.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-03.csv']

# spin up cluster

In [39]:
from dask.distributed import get_task_stream
from dask.distributed import Client
from dask_kubernetes import KubeCluster

In [42]:
#tag::make_dask_k8s_client[]
import dask
from dask.distributed import Client

from dask_kubernetes import KubeCluster, make_pod_spec
worker_template = make_pod_spec(image='holdenk/dask:latest',
                         memory_limit='8G', memory_request='8G',
                         cpu_limit=1, cpu_request=1, extra_container_config={ "imagePullPolicy": "Always" })
scheduler_template = make_pod_spec(image='holdenk/dask:latest',
                         memory_limit='4G', memory_request='4G',
                         cpu_limit=1, cpu_request=1, extra_container_config={ "imagePullPolicy": "Always" })


In [43]:
cluster = KubeCluster(pod_template = worker_template, scheduler_pod_template = scheduler_template)
cluster.adapt(minimum=1)    # or create and destroy workers dynamically based on workload
from dask.distributed import Client
client = Client(cluster)
#end::make_dask_k8s_client[]

Creating scheduler pod on cluster. This may take some time.


distributed.deploy.adaptive - INFO - Adaptive scaling started: minimum=1 maximum=inf
/opt/conda/lib/python3.8/site-packages/distributed/client.py:1123: VersionMismatchWarning: Mismatched versions found

+-------------+-----------+-----------+---------+
| Package     | client    | scheduler | workers |
+-------------+-----------+-----------+---------+
| dask        | 2021.09.1 | 2022.02.0 | None    |
| distributed | 2021.09.1 | 2022.02.0 | None    |
+-------------+-----------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [44]:
client

Connection method: Cluster object,Cluster type: dask_kubernetes.KubeCluster
Dashboard: http://dask-jovyan-c460a5e1-5.default:8787/status,
Dashboard: http://dask-jovyan-c460a5e1-5.default:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.42.4.85:8786,Workers: 0
Dashboard: http://10.42.4.85:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [26]:
import re
pattern = '/[^\/]*\.csv$/gm'

csv_urls = [re.findall(r"[^\/]*\.csv$",csv_url)[0] for csv_url in yellow]

csv_urls = yellow

In [77]:
@dask.delayed
def read_csv(url):
    try:
        df = dd.read_csv(url, dtype={'tolls_amount': 'float64',
                                     'tip_amount': 'float64',
                                     'Tip_Amt': 'float64',
                                     'Tolls_Amt': 'float64',
                                     'surcharge': 'float64',
                                     'store_and_fwd_flag': 'object'})
        return df
    except Exception as e:
        print(f"failed, {url}")
        print(e)

In [78]:
csv_urls

['https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-01.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-02.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-03.csv']

In [87]:
x1 = read_csv('https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-01.csv')

In [89]:
x2 = x1.compute()

distributed.protocol.pickle - INFO - Failed to deserialize <memory at 0x7ece9fb400>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/distributed/protocol/pickle.py", line 75, in loads
    return pickle.loads(x)
AttributeError: Can't get attribute '_unpickle_block' on <module 'pandas._libs.internals' from '/opt/conda/lib/python3.8/site-packages/pandas/_libs/internals.cpython-38-aarch64-linux-gnu.so'>
distributed.protocol.core - CRITICAL - Failed to deserialize
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/distributed/protocol/core.py", line 111, in loads
    return msgpack.loads(
  File "msgpack/_unpacker.pyx", line 194, in msgpack._cmsgpack.unpackb
  File "/opt/conda/lib/python3.8/site-packages/distributed/protocol/core.py", line 103, in _decode_default
    return merge_and_deserialize(
  File "/opt/conda/lib/python3.8/site-packages/distributed/protocol/serialize.py", line 475, in merge_and_deserialize
    return dese

AttributeError: Can't get attribute '_unpickle_block' on <module 'pandas._libs.internals' from '/opt/conda/lib/python3.8/site-packages/pandas/_libs/internals.cpython-38-aarch64-linux-gnu.so'>

In [115]:
# dfs = [delayed(read_csv)(filename) for filename in csv_urls]
# dfs = [read_csv(filename) for filename in csv_urls]
# dfs = [delayed(read_csv(filename)) for filename in csv_urls]
# dfs = [dd.read_csv(filename, dtype={'tolls_amount': 'float64',
#                                      'tip_amount': 'float64',
#                                      'Tip_Amt': 'float64',
#                                      'Tolls_Amt': 'float64',
#                                      'surcharge': 'float64',
#                                      'store_and_fwd_flag': 'object'}) for filename in csv_urls]
dfs = [delayed(dd.read_csv(filename, dtype={'tolls_amount': 'float64',
                                     'tip_amount': 'float64',
                                     'Tip_Amt': 'float64',
                                     'Tolls_Amt': 'float64',
                                     'surcharge': 'float64',
                                     'store_and_fwd_flag': 'object'})) for filename in csv_urls]

In [ ]:
ddf = dd.from_delayed(dfs)

distributed.deploy.adaptive - INFO - Retiring workers [1]


In [ ]:
ddf

In [92]:
dfs

[Dask DataFrame Structure:
                vendor_id pickup_datetime dropoff_datetime passenger_count trip_distance pickup_longitude pickup_latitude rate_code store_and_fwd_flag dropoff_longitude dropoff_latitude payment_type fare_amount surcharge  mta_tax tip_amount tolls_amount total_amount
 npartitions=39                                                                                                                                                                                                                                                           
                   object          object           object           int64       float64          float64         float64     int64             object           float64          float64       object     float64   float64  float64    float64      float64      float64
                      ...             ...              ...             ...           ...              ...             ...       ...                ...               ...    

In [93]:
dfs

[Dask DataFrame Structure:
                vendor_id pickup_datetime dropoff_datetime passenger_count trip_distance pickup_longitude pickup_latitude rate_code store_and_fwd_flag dropoff_longitude dropoff_latitude payment_type fare_amount surcharge  mta_tax tip_amount tolls_amount total_amount
 npartitions=39                                                                                                                                                                                                                                                           
                   object          object           object           int64       float64          float64         float64     int64             object           float64          float64       object     float64   float64  float64    float64      float64      float64
                      ...             ...              ...             ...           ...              ...             ...       ...                ...               ...    

In [95]:
# df = dd.from_delayed(dfs)

# define benchmarking

In [99]:
import collections
dask_benchmarks = collections.defaultdict(dict)
# benchmarks = {"task1" : {"stat1": val, "stat2": val}}

In [108]:
all_tasks = []
# sum, then means = simple mapreduce
# def read_to_basic_ETL(df = None):
#     df = dd.read_parquet(
#     "./tmp/", 
#     storage_options={"anon": True, 'use_ssl': True})    
#     return (df.fare_amount + df.tip_amount).mean().compute()
# all_tasks.append(read_to_basic_ETL)
# # counts of values seen = simple map, groupby, reduce

def basic_ETL(df = None):
    return (df.fare_amount + df.tip_amount).mean().compute()
all_tasks.append(basic_ETL)
# counts of values seen = simple map, groupby, reduce


def count_values(df):
    return df.fare_amount.value_counts().compute()
all_tasks.append(count_values)

# cpu heavy arithmetic : mapreduce
def complicated_arithmetic_operation(df):
    theta_1 = df.pickup_longitude
    phi_1 = df.pickup_latitude
    theta_2 = df.dropoff_longitude
    phi_2 = df.dropoff_latitude
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
           + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    ret = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))
    return ret.compute()
all_tasks.append(complicated_arithmetic_operation)

def groupby_statistics(df):
    return df.groupby(by='passenger_count').agg(
      {
        'total_amount': ['mean', 'std'], 
        'tip_amount': ['mean', 'std']
      }
    ).compute()
all_tasks.append(complicated_arithmetic_operation)

# join two datasets
def join_data(df):
    return dd.merge(df, other, left_index=True, right_index=True).compute()
all_tasks.append(complicated_arithmetic_operation)



In [67]:
def benchmark(f, df, benchmarks, task_name, **kwargs):
    """Benchmark the given function against the given DataFrame.
    
    Parameters
    ----------
    f: function to benchmark
    df: data frame
    benchmarks: container for benchmark results
    name: task name
    
    Returns
    -------
    Duration (in seconds) of the given operation
    """
    ret_benchmark_vals = {}
    print(f"{task_name} start")
    with get_task_stream(plot='save', filename="task-stream.html") as ts:
        start_time = time.time()
        ret = f(df, **kwargs)
        ret_benchmark_vals['raw_duration'] = time.time() - start_time
        ret_benchmark_vals['history'] = ts.data
    benchmarks[task_name] = ret_benchmark_vals
    print(f"{task_name} took: {benchmarks[task_name].get('raw_duration')} seconds")
    return benchmarks[task_name].get("raw_duration")

In [113]:
type(df)

list

In [97]:
df = dfs

In [111]:
[benchmark(task, df=df, benchmarks = dask_benchmarks, task_name = task.__name__) for task in all_tasks]

basic_ETL start


AttributeError: 'list' object has no attribute 'fare_amount'